In [ ]:
# Imports and configuration

import model
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format = 'retina'
sns.set_style("ticks")
plt.rc("axes.spines", top=False, right=False)

In [ ]:
# Model Parameters

num_agents = 200
ticks = 2000
match_on_order = True

In [ ]:
# Run the model here

havven = model.Havven(num_agents, match_on_order=match_on_order)
for _ in tqdm(range(ticks)):
    havven.step()

In [ ]:
# Collect and plot the data

df = havven.datacollector.get_model_vars_dataframe()
df[["Curit Demand", "Curit Supply"]].plot()
df[["Nomin Demand", "Nomin Supply"]].plot()
df[["Fiat Demand", "Fiat Supply"]].plot()
df[["Profit %"]].plot()
df[["Escrowed Curits", "Nomins"]].plot()
df[["Fee Pool", "Fees Distributed"]].plot()

plt.show()